In [58]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
#from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
#from dmba import classificationSummary, gainsChart, liftChart
#from dmba.metric import AIC_score

In [59]:
bank_df = pd.read_csv("C:/Users/sekha/Desktop/BUSI 650 SP22/UniversalBank.csv")
bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)
bank_df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [60]:
bank_df = pd.read_csv("C:/Users/sekha/Desktop/BUSI 650 SP22/UniversalBank.csv")
bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)
bank_df.columns = [c.replace(' ', '_') for c in bank_df.columns]
bank_df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [61]:
# Treat education as categorical, convert to dummy variables
bank_df['Education'] = bank_df['Education'].astype('category')
new_categories = {1: 'Undergrad', 2: 'Graduate', 3: 'Advanced/Professional'}
bank_df.Education.cat.rename_categories(new_categories, inplace=True)
bank_df = pd.get_dummies(bank_df, prefix_sep='_', drop_first=True)
bank_df.head()

C:\Users\sekha\anaconda3\lib\site-packages\pandas\core\arrays\categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.rename_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_Graduate,Education_Advanced/Professional
0,25,1,49,4,1.6,0,0,1,0,0,0,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,0,1,1,0


In [63]:
#code for fitting a logistic regression model
#bank_df = pd.read_csv("C:/Users/sekha/Desktop/BUSI 650 SP22/UniversalBank.csv")
#bank_df.drop(columns=['ID', 'ZIP Code'], inplace=True)
#bank_df.columns = [c.replace(' ', '_') for c in bank_df.columns]

# Treat education as categorical, convert to dummy variables
#bank_df['Education'] = bank_df['Education'].astype('category')
#new_categories = {1: 'Undergrad', 2: 'Graduate', 3: 'Advanced/Professional'}
#bank_df.Education.cat.rename_categories(new_categories, inplace=True)
#bank_df = pd.get_dummies(bank_df, prefix_sep='_', drop_first=True)

y = bank_df['Personal_Loan']
X = bank_df.drop(columns=['Personal_Loan'])

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a logistic regression (set penalty=l2 and C=1e42 to avoid regularization)
#The theory says that in order to achieve prediction consistency, the penalty parameter
#should be kept constant as the number of samples grow.
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
# liblinear — Library for Large Linear Classification.
#C is a regularization parameter that controls the trade off
#between the achieving a low training error and a low testing error
logit_reg.fit(train_X, train_y)

print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())

#print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df = len(train_X.columns) + 1))

#Output

intercept  -12.105194006575854
            Age  Experience    Income    Family     CCAvg  Mortgage  \
coeff -0.051667    0.052952  0.058676  0.609565  0.240421   0.00101   

       Securities_Account  CD_Account    Online  CreditCard  \
coeff           -1.030108    3.648152 -0.678944   -0.961048   

       Education_Graduate  Education_Advanced/Professional  
coeff             4.19873                         4.358777  


In [64]:
#code for using logistic regression to generate predicted probabilities
logit_reg_pred = logit_reg.predict(valid_X)
logit_reg_proba = logit_reg.predict_proba(valid_X)
logit_result = pd.DataFrame({'actual': valid_y, 
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })
			     
# display four different cases
interestingCases = [2764, 932, 2721, 702]
print(logit_result.loc[interestingCases])

      actual      p(0)      p(1)  predicted
2764       0  0.975938  0.024062          0
932        0  0.331932  0.668068          1
2721       1  0.031478  0.968522          1
702        1  0.985949  0.014051          0


In [68]:
interestingCases = [2768]
print(logit_result.loc[interestingCases])

      actual      p(0)      p(1)  predicted
2768       0  0.950192  0.049808          0


In [71]:
interestingCases = [1]
print(logit_result.loc[interestingCases])

   actual      p(0)      p(1)  predicted
1       0  0.999965  0.000035          0


In [100]:
#confusion matrics for train
from sklearn.metrics import accuracy_score
logit_reg_pred = logit_reg.predict(train_X)
accuracy = accuracy_score(train_y,logit_reg_pred)
print(accuracy)
confusion_matrix(train_y,logit_reg_pred)

0.9606666666666667


array([[2685,   28],
       [  90,  197]], dtype=int64)

In [102]:
#confusion matrixs for test data
from sklearn.metrics import accuracy_score
#logit_reg_pred=logit_reg.predict(valid_x)
logit_reg_pred = logit_reg.predict(valid_X)
accuracy = accuracy_score(valid_y,logit_reg_pred)
print(accuracy)
confusion_matrix(valid_y,logit_reg_pred)

0.9595


array([[1791,   16],
       [  65,  128]], dtype=int64)